https://stackoverflow.com/questions/24356683/apply-grouped-model-back-onto-data

In [6]:
library(dplyr)
library(broom)
library(tibble)
library(tidyr)
library(purrr)
library(caret)

In [15]:
dim(iris)
head(iris)

[1] 150   5

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa


In [27]:
set.seed(13579)
trainIndex <- createDataPartition(iris$Species, p = .8, 
                                  list = FALSE, 
                                  times = 1)

irisTrain <- iris[ trainIndex,] %>% tbl_df
irisTest  <- iris[-trainIndex,] %>% tbl_df

irisTrain %>% group_by(Species) %>% count()
irisTest %>% group_by(Species) %>% count()

Species,n
setosa,40
versicolor,40
virginica,40


Species,n
setosa,10
versicolor,10
virginica,10


In [29]:
irisTrain %>%
group_by(Species) %>%
do(mod = lm(Sepal.Length ~ Sepal.Width, data = .)) -> models

print(models)

Source: local data frame [3 x 2]
Groups: <by row>

# A tibble: 3 x 2
     Species      mod
*     <fctr>   <list>
1     setosa <S3: lm>
2 versicolor <S3: lm>
3  virginica <S3: lm>


In [83]:
irisTest %>% 
nest(-Species) %>%
inner_join(models, by="Species") -> x

x %>% print

x %>% mutate(pred = predict(mod, as.data.frame(data))) -> y
y %>% print

# A tibble: 3 x 3
     Species              data      mod
      <fctr>            <list>   <list>
1     setosa <tibble [10 x 4]> <S3: lm>
2 versicolor <tibble [10 x 4]> <S3: lm>
3  virginica <tibble [10 x 4]> <S3: lm>
# A tibble: 3 x 4
     Species              data      mod       pred
      <fctr>            <list>   <list>     <list>
1     setosa <tibble [10 x 4]> <S3: lm> <dbl [10]>
2 versicolor <tibble [10 x 4]> <S3: lm> <dbl [10]>
3  virginica <tibble [10 x 4]> <S3: lm> <dbl [10]>
